In [1]:
from pygle import network
import pandas as pd
import numpy as np
#La url donde está el archivo de configuración de pygle es C:\Users\Guille\Anaconda3\envs\py27\lib\site-packages\pygle\config.pyc

In [6]:
# Copyright (c) 2017 Jamie Bull - oCo Carbon Ltd
# =======================================================================
#  Distributed under the MIT License.
# =======================================================================
"""
Sniff local WiFi networks in order to look them up on WiGLE.

"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from platform import system
import re
import subprocess
import time

import requests

from pygle import network


scan_cmd = {'windows': 'netsh wlan show networks mode=Bssid', 
            'linux': 'nmcli -t -f bssid dev wifi list',  # untested
            'osx': '/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport -s',  # untested
            }
extremo_norte =  40.2149;   
extremo_sur = 40.2193;
extremo_este = -3.8634;    
extremo_oeste = -3.8484;
regexp = re.compile('MOVISTAR');
def local_bssids():
    """Get a list of available wireless access points.
    """
    res = subprocess.check_output("nmcli -t -f bssid dev wifi list",
                                  shell=True)
   
    res= res.replace("\\" , "")
    
    pattern = re.compile(r'(?:[0-9a-fA-F]:?){12}')
    bssids = pattern.findall(res)
   
    return set(bssids)


def geolocate():
    """Locate a device using locally available wireless access points.
    """
    lats = []
    longs = []
    for bssid in local_bssids():
        print("BSSID:", bssid)
        lat, lng = geolocate_wigle(bssid)  # can be replaced with other geocoding sources
        if lat and lng:
            lats.append(lat)
            longs.append(lng)
        time.sleep(0.1)
    if lats and longs:
        lat = sum(lats) / len(lats)
        lng = sum(longs) / len(longs)
        return lat, lng
    else:
        return "No geolocation possible"


def geolocate_wigle(bssid):
    """Search WiGLE for a BSSID and return lat/lng.
    """
    res = network.search(netid=bssid)
    if res['success'] and res['resultCount']:
        lat = res['results'][0]['trilat']
        lng = res['results'][0]['trilong']
    else:
        print(res)
        lat, lng = None, None
    return lat, lng

def network_data_parser(long_min, lat_min, long_max, lat_max):
    """Búsqueda dentro de WiGLE de redes dentro de una localización y devuelve toda la info de la BBDD
    """
    lats = []
    longs = []
    ssids = []
    netpoint = []
    network_map = []
    res = network.search(latrange1=lat_min, latrange2=lat_max, longrange1=long_min, longrange2=long_max, ssidlike='WLAN%')
    print(res['totalResults']);
    if res['success'] and res['resultCount']:
        for net in res['results']:
            comment = net['comment']
            qos = net['qos']
            transid = net['transid']
            city = net['city']
            ssid = net['ssid']
            encryption = net['encryption']
            wep = net['wep']
            bcninterval = net['bcninterval']
            dhcp = net['dhcp']
            typ = net['type']
            channel = net['channel']
            lng = net['trilong']
            paynet = net['paynet']
            userfound = net['userfound']
            housenumber = net['housenumber']
            lasttime = net['lasttime']
            firsttime = net['firsttime']
            name = net['name']
            lastupdt = net['lastupdt']
            country = net['country']
            region = net['region']
            netid = net['netid']
            freenet = net['freenet']
            lat = net['trilat']
            road = net['road']
            
            netpoint= [comment, qos, transid, city, ssid, encryption, wep, bcninterval, dhcp, typ, channel, lng, 
                       paynet, userfound, housenumber, lasttime, firsttime, name, lastupdt, country, region, 
                       netid, freenet, lat, road]
            network_map.append(netpoint)
        
           
        return res['totalResults']
    else:
        print(res)
        lat, lng = None, None
        return lat, lng
#if __name__ == "__main__":
    #print("WiGLE")
    

In [7]:
sumatorio_aprox_total_chamberi =network_data_parser(-3.7199481, 40.4248479, -3.6884365, 40.447132);
sumatorio_aprox_total_chamartin =network_data_parser(-3.6917412, 40.4373224, -3.6604484, 40.48406);

11696
17523


In [7]:
"""(-3.7229418, 40.3821877, -3.6732365, 40.4140254) 'arganzuela.geojson' Hecho
(-3.6279521, 40.4464811, -3.5250873, 40.5132684) 'barajas.geojson' Hecho
(-3.7713925, 40.3572614, -3.7046325, 40.4055319) 'carabanchel.geojson' Hecho
(-3.7229123, 40.405002, -3.6904647, 40.4306263) 'centro.geojson' Hecho
(-3.6917412, 40.4373224, -3.6604484, 40.48406) 'chamartin.geojson' Hecho
(-3.7199481, 40.4248479, -3.6884365, 40.447132) 'chamberi.geojson' Hecho
(-3.6743987, 40.4130616, -3.6203687, 40.4841782) 'ciudad-linea.geojson' Hecho
(-3.8889539, 40.4684881, -3.6019947, 40.6437293) 'fuencarral-el-pardo.geojson' Hecho
(-3.6734578, 40.4485438, -3.6028416, 40.5131585) 'hortaleza.geojson' Hecho
(-3.8351908, 40.3587453, -3.7217887, 40.4172845) 'latina.geojson' Hecho
(-3.8383521, 40.3992355, -3.7082704, 40.4797786) 'moncloa-aravaca.geojson' Hecho
(-3.6653928, 40.3925625, -3.6203687, 40.4186705) 'moratalaz.geojson' Hecho
(-3.6895732, 40.360849, -3.6217467, 40.4063258) 'puente-de-vallecas.geojson' Hecho
(-3.6941294, 40.3945135, -3.6587726, 40.4217289) 'retiro.geojson' Hecho
(-3.6930297, 40.4186705, -3.65879, 40.4445817) 'salamanca.geojson' Hecho
(-3.648042, 40.4076791, -3.5349879, 40.4502028) 'san-blas.geojson' Hecho
(-3.7133277, 40.4462118, -3.6868496, 40.4753456) 'tetuan.geojson' Hecho
(-3.7211547, 40.3614412, -3.6817179, 40.3950807) 'usera.geojson' Hecho
(-3.6297953, 40.3557577, -3.5179163, 40.4204795) 'vicalvaro.geojson' Hecho
(-3.6819717, 40.3121885, -3.5534844, 40.3908047) 'villa-de-vallecas.geojson' Hecho
(-3.7251721, 40.3207453, -3.657359, 40.3654057) 'villaverde.geojson' Hecho



, ssidlike='MOVISTAR%'
sumatorio=0;
for bounds in centro:
    prueba =network_data_parser(bounds[2], bounds[4], bounds[1], bounds[3]);
    if type(prueba) == int:
        sumatorio = prueba + sumatorio;"""

sumatorio_aprox_wlan_arganzuela =network_data_parser(-3.7229418, 40.3821877, -3.6732365, 40.4140254);
sumatorio_aprox_wlan_barajas =network_data_parser(-3.6279521, 40.4464811, -3.5250873, 40.5132684);
sumatorio_aprox_wlan_carabanchel =network_data_parser(-3.7713925, 40.3572614, -3.7046325, 40.4055319);
sumatorio_aprox_wlan_centro =network_data_parser(-3.7229123, 40.405002, -3.6904647, 40.4306263);
sumatorio_aprox_wlan_chamartin =network_data_parser(-3.6917412, 40.4373224, -3.6604484, 40.48406);
sumatorio_aprox_wlan_chamberi =network_data_parser(-3.7199481, 40.4248479, -3.6884365, 40.447132);
sumatorio_aprox_wlan_ciudad_lineal =network_data_parser(-3.6743987, 40.4130616, -3.6203687, 40.4841782);
sumatorio_aprox_wlan_fuencarral =network_data_parser(-3.8889539, 40.4684881, -3.6019947, 40.6437293);
sumatorio_aprox_wlan_hortaleza =network_data_parser(-3.6734578, 40.4485438, -3.6028416, 40.5131585);
sumatorio_aprox_wlan_latina =network_data_parser(-3.8351908, 40.3587453, -3.7217887, 40.4172845);
sumatorio_aprox_wlan_moncloa =network_data_parser(-3.8383521, 40.3992355, -3.7082704, 40.4797786);
sumatorio_aprox_wlan_moratalaz =network_data_parser(-3.6653928, 40.3925625, -3.6203687, 40.4186705);
sumatorio_aprox_wlan_puente_de_vallecas =network_data_parser(-3.6895732, 40.360849, -3.6217467, 40.4063258);
sumatorio_aprox_wlan_retiro =network_data_parser(-3.6941294, 40.3945135, -3.6587726, 40.4217289);
sumatorio_aprox_wlan_salamanca =network_data_parser(-3.6930297, 40.4186705, -3.65879, 40.4445817);
sumatorio_aprox_wlan_san_blas =network_data_parser(-3.648042, 40.4076791, -3.5349879, 40.4502028);
sumatorio_aprox_wlan_tetuan =network_data_parser(-3.7133277, 40.4462118, -3.6868496, 40.4753456);
sumatorio_aprox_wlan_usera =network_data_parser(-3.7211547, 40.3614412, -3.6817179, 40.3950807);
sumatorio_aprox_wlan_vicalvaro =network_data_parser(-3.6297953, 40.3557577, -3.5179163, 40.4204795);
sumatorio_aprox_wlan_villa_de_vallecas =network_data_parser(-3.6819717, 40.3121885, -3.5534844, 40.3908047);
sumatorio_aprox_wlan_villaverde =network_data_parser(-3.7251721, 40.3207453, -3.657359, 40.3654057);


0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
4
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
16
3
5
5
2
3
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
9
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
2
3
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, u'first': 1}
0
{u'search_after': None, u'last': 0, u'success': True, u'results': [], u'resultCount': 0, u'totalResults': 0, 

In [12]:
sumatorio_aprox_orange_arganzuela = 6757;
sumatorio_aprox_orange_barajas = 775;
sumatorio_aprox_orange_carabanchel =2456
sumatorio_aprox_orange_centro =9012
sumatorio_aprox_orange_chamartin =5589
sumatorio_aprox_orange_chamberi =3963
sumatorio_aprox_orange_ciudad_lineal =9060
sumatorio_aprox_orange_fuencarral =3904
sumatorio_aprox_orange_hortaleza =1814
sumatorio_aprox_orange_latina =2674
sumatorio_aprox_orange_moncloa =5576
sumatorio_aprox_orange_moratalaz =705
sumatorio_aprox_orange_puente_de_vallecas =2425
sumatorio_aprox_orange_retiro =3128
sumatorio_aprox_orange_salamanca =7416
sumatorio_aprox_orange_san_blas =2498
sumatorio_aprox_orange_tetuan =2646
sumatorio_aprox_orange_usera =2441
sumatorio_aprox_orange_vicalvaro =1419
sumatorio_aprox_orange_villa_de_vallecas =795
sumatorio_aprox_orange_villaverde =1963

In [8]:
sumatorio_aprox_vodafone_arganzuela = 6758;
sumatorio_aprox_vodafone_barajas = 943;
sumatorio_aprox_vodafone_carabanchel =1826
sumatorio_aprox_vodafone_centro =9222
sumatorio_aprox_vodafone_chamartin =6726
sumatorio_aprox_vodafone_chamberi =4609
sumatorio_aprox_vodafone_ciudad_lineal =10178
sumatorio_aprox_vodafone_fuencarral =3637
sumatorio_aprox_vodafone_hortaleza =2109
sumatorio_aprox_vodafone_latina =1897
sumatorio_aprox_vodafone_moncloa =5620
sumatorio_aprox_vodafone_moratalaz =439
sumatorio_aprox_vodafone_puente_de_vallecas =2356
sumatorio_aprox_vodafone_retiro =3068
sumatorio_aprox_vodafone_salamanca =9026
sumatorio_aprox_vodafone_san_blas =2422
sumatorio_aprox_vodafone_tetuan =2211
sumatorio_aprox_vodafone_usera =1713
sumatorio_aprox_vodafone_vicalvaro =1233
sumatorio_aprox_vodafone_villa_de_vallecas =930
sumatorio_aprox_vodafone_villaverde =1150

In [79]:
sumatorio_aprox_villaverde =network_data_parser(-3.7251721, 40.3207453, -3.657359, 40.3654057);

16711


In [81]:
sumatorio_aprox_movistar_arganzuela = 9496;
sumatorio_aprox_arganzuela = 78175;
sumatorio_aprox_movistar_barajas = 2273;
sumatorio_aprox_barajas = 31771;

In [8]:
sumatorio_movistar_arganzuela#5074
sumatorio_total_arganzuela#41110
sumatorio_total_carabanchel#14096
sumatorio_movistar_carabanchel#577
sumatorio_movistar_centro#16133
sumatorio_total_centro#132066

5074

In [1]:
sumatorio_aprox_movistar_arganzuela = 9496
sumatorio_aprox_arganzuela = 78175
sumatorio_aprox_movistar_barajas = 2273
sumatorio_aprox_barajas=31771
sumatorio_aprox_movistar_carabanchel=1751
sumatorio_aprox_carabanchel=25548
sumatorio_aprox_movistar_centro=17698
sumatorio_aprox_centro=141484
sumatorio_aprox_movistar_chamartin=16716
sumatorio_aprox_chamartin=92249
sumatorio_aprox_movistar_chamberi=16716
sumatorio_aprox_chamberi=92249
sumatorio_aprox_movistar_ciudad_lineal=21109
sumatorio_aprox_ciudad_lineal=119216
sumatorio_aprox_movistar_fuencarral=6119
sumatorio_aprox_fuencarral=65182
sumatorio_aprox_movistar_hortaleza=5139
sumatorio_aprox_hortaleza=37212
sumatorio_aprox_movistar_latina=2277
sumatorio_aprox_latina=27743
sumatorio_aprox_movistar_moncloa=13296
sumatorio_aprox_moncloa=88411
sumatorio_aprox_movistar_moratalaz=584
sumatorio_aprox_moratalaz=6847
sumatorio_aprox_movistar_puente_de_vallecas=3009
sumatorio_aprox_puente_de_vallecas=27738
sumatorio_aprox_movistar_retiro=5959
sumatorio_aprox_retiro=49485
sumatorio_aprox_movistar_salamanca=23260
sumatorio_aprox_salamanca=136075
sumatorio_aprox_movistar_san_blas=3979
sumatorio_aprox_san_blas=39037
sumatorio_aprox_movistar_tetuan=3728
sumatorio_aprox_tetuan=42517
sumatorio_aprox_movistar_usera=2670
sumatorio_aprox_usera=21509
sumatorio_aprox_movistar_vicalvaro=1432
sumatorio_aprox_vicalvaro=16741
sumatorio_aprox_movistar_villa_de_vallecas=690
sumatorio_aprox_villa_de_vallecas=11446
sumatorio_aprox_movistar_villaverde=1931
sumatorio_aprox_villaverde=16711



